## Data

In [ ]:
from random import random
import numpy as np

```C
int N;
vector[N] observed_counts;
vector[N] background_counts;
vector[N] background_errors;
vector[N] expected_model_counts;
array[N] int idx_background_zero;
array[N] int idx_background_nonzero;
```

In [ ]:
N = 20
observed_counts = np.array([600 * random() for _ in range(N)])
background_counts = np.array([3 * random() for _ in range(N)])
background_errors = np.array([random() for _ in range(N)])
expected_model_counts = np.array([1e-5 * random() for _ in range(N)])

In [ ]:
idx_background_zero = np.zeros(N)
idx_background_nonzero = np.zeros(N)

N_bkg_zero = 0
N_bkg_nonzero = 0
for i in range(N):
    if background_counts[i] == 0:
        N_bkg_zero += 1
        idx_background_zero[i] = N_bkg_zero
    else:
        N_bkg_nonzero += 1
        idx_background_nonzero[i] = N_bkg_nonzero
    # if random() < 0.1:
    #     background_counts[i] = 0

In [ ]:
data = dict(
    N=N,
    observed_counts=observed_counts,
    background_counts=background_counts,
    background_errors=background_errors,
    expected_model_counts=expected_model_counts,
    idx_background_zero=np.array(idx_background_zero).astype(np.int64),
    idx_background_nonzero=np.array(idx_background_nonzero).astype(np.int64),
    N_bkg_zero=N_bkg_zero,
    N_bkg_nonzero=N_bkg_nonzero,
)

## 3ML

In [ ]:
from math import sqrt, log, pi, lgamma
from threeML.utils.statistics.likelihood_functions import xlogy_one

In [ ]:
def pgstat_3ml(
    observed_counts, background_counts, background_error, expected_model_counts
):

    # This loglike assume Gaussian errors on the background and Poisson uncertainties on the

    # observed counts. It is a profile likelihood.
    n = background_counts.shape[0]

    log_likes = np.empty(n, dtype=np.float64)
    b = np.empty(n, dtype=np.float64)

    for idx in range(n):

        MB = background_counts[idx] + expected_model_counts[idx]
        s2 = background_error[idx] * background_error[idx]  # type: np.ndarray

        b[idx] = 0.5 * (
            sqrt(MB * MB - 2 * s2 * (MB - 2 * observed_counts[idx]) + s2 * s2)
            + background_counts[idx]
            - expected_model_counts[idx]
            - s2
        )  # type: np.ndarray

        # Now there are two branches: when the background is 0 we are in the normal situation of a pure
        # Poisson likelihood, while when the background is not zero we use the profile likelihood

        # NOTE: bkgErr can be 0 only when also bkgCounts = 0
        # Also it is evident from the expression above that when bkgCounts = 0 and bkgErr=0 also b=0

        # Let's do the branch with background > 0 first

        if background_counts[idx] > 0:

            log_likes[idx] = (
                -((b[idx] - background_counts[idx]) ** 2) / (2 * s2)
                + observed_counts[idx] * log(b[idx] + expected_model_counts[idx])
                - b[idx]
                - expected_model_counts[idx]
                - lgamma(observed_counts[idx] + 1)
                - 0.5 * log(2 * pi)
                - log(background_error[idx])
            )

        # Let's do the other branch

        else:

            # the 1e-100 in the log is to avoid zero divisions
            # This is the Poisson likelihood with no background
            log_likes[idx] = (
                xlogy_one(observed_counts[idx], expected_model_counts[idx])
                - expected_model_counts[idx]
                - lgamma(observed_counts[idx] + 1)
            )

    return log_likes

In [ ]:
for i in range(N):
    pgstat_3ml_list = pgstat_3ml(background_counts=background_counts, background_error=background_errors, observed_counts=observed_counts, expected_model_counts=expected_model_counts)
pgstat_3ml_list

## Stan

In [ ]:
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from zusammen.stan_models.stan_model import get_model
from zusammen import AnalysisBuilder, DataSet
from zusammen.spectral_plot import display_posterior_model_counts

In [ ]:
mt = get_model("pgstat_test")
mt.build_model()
mt.model.sample(data=data, fixed_param=True, show_console=True)

## Comparison
3ML:
```python
array([ -167.7694011 , -1994.66237653,   -98.24866785, -1637.35182234,
       -1154.07910362,  -316.68887444,  -530.40550602,  -934.71597183,
       -1542.09324413, -1241.30541153,   -30.22217146,   -82.57800668,
        -725.45924881,  -119.52227875, -1638.64787183,  -802.58327595,
       -1462.02490065, -1786.18362936,  -918.10856812,  -480.09451077])
```
Stan:
```C
[-167.769,-1994.66,-98.2487,-1637.35,-1154.08,
 -316.689,-530.406,-934.716,-1542.09,-1241.31,-30.2222,
 -82.578,-725.459,-119.522,-1638.65,-802.583,-1462.02,
 -1786.18,-918.109,-480.095]
```

In [ ]:
for i, j in zip(
    [
        -167.769, -1994.66, -98.2487, -1637.35, -1154.08, -316.689, -530.406, -934.716, -1542.09, -1241.31, -30.2222, -82.578, -725.459, -119.522, -1638.65, -802.583, -1462.02, -1786.18, -918.109, -480.095,
    ],
    [
        -167.7694011, -1994.66237653, -98.24866785, -1637.35182234, -1154.07910362, -316.68887444, -530.40550602, -934.71597183, -1542.09324413, -1241.30541153, -30.22217146, -82.57800668, -725.45924881, -119.52227875, -1638.64787183, -802.58327595, -1462.02490065, -1786.18362936, -918.10856812, -480.09451077,
    ],
):
    print(abs(i - j) < 0.01)